In [33]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [34]:
df = pd.read_csv('job_data_cleaned.csv')

In [35]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min_salary', 'max_salary', 'avg_salary', 'state', 'age', 'python',
       'r_studio', 'spark', 'aws', 'tableau', 'excel', 'ml'],
      dtype='object')

In [36]:
def job_title_cleaning(title):
    """
    input: Job titles as a string
    output: A string containing one of the five basic categories or na if none apply 
    """
    if 'data scientist' in title.lower():
        return 'data_scientist'
    elif 'data engineer' in title.lower():
        return "data_engineer"
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'mle'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'

In [37]:
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower():
        return 'junior'
    else:
        return 'na'
    

In [40]:
# Simplify job description into 5 categories
df['job_simplified'] = df['Job Title'].apply(job_title_cleaning)

# Extract seniority information from jobt title
df['seniority'] = df['Job Title'].apply(seniority)

In [50]:
# Check number of jobs in each category
df.job_simplified.value_counts()

data_scientist    503
na                171
data_engineer      57
analyst            37
mle                19
manager             9
director            4
Name: job_simplified, dtype: int64

In [51]:
# Check seniority distribution
df.seniority.value_counts()

na        600
senior    200
Name: seniority, dtype: int64

In [46]:
# Job description length
df['desc_length'] = df['Job Description'].apply(lambda x: len(x))

In [56]:
# 
df.desc_length.value_counts()

2495    4
3361    3
8173    3
4286    3
3204    3
       ..
1673    1
2696    1
1671    1
3284    1
3073    1
Name: desc_length, Length: 679, dtype: int64

In [49]:
# We didnt find any competitors while scraping
df.Competitors.unique()

array([-1], dtype=int64)

In [65]:
# 
df.describe()

,Unnamed: 0,Rating,Headquarters,Competitors,min_salary,max_salary,avg_salary,age,python,r_studio,spark,aws,tableau,excel,ml,desc_length
count,800.0000,800.000000,800.0,800.0,800.00000,800.000000,800.000000,680.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000
mean,399.5000,3.783125,-1.0,-1.0,76.50250,142.573750,109.538125,51.302941,0.082500,0.002500,0.036250,0.052500,0.028750,0.252500,0.253750,3885.537500
std,231.0844,1.151525,0.0,0.0,15.07254,27.060763,20.621009,50.563913,0.275297,0.049969,0.187029,0.223173,0.167208,0.434718,0.435428,1843.944127
min,0.0000,-1.000000,-1.0,-1.0,48.00000,97.000000,73.500000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,60.000000
25%,199.7500,3.700000,-1.0,-1.0,65.00000,127.000000,94.500000,16.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2695.000000
50%,399.5000,4.000000,-1.0,-1.0,74.00000,136.000000,108.000000,34.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3596.500000
75%,599.2500,4.300000,-1.0,-1.0,90.00000,158.000000,122.000000,73.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,4766.500000
max,799.0000,5.000000,-1.0,-1.0,104.00000,196.000000,146.500000,275.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,18466.000000
